![title](https://i.imgur.com/NsCStAG.png)
<p style="text-align:center">
    Este é um notebook de apresentação da resolução do business case de Data Science fornecido pela Loft.<br>
    <span style="font-size:10px;">Desenvolvido por Pedro Almeida.</span>
</p>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import cluster
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
import xgboost as xgb

# Explorando os dados

In [ ]:
df = pd.read_csv('../input/base-imoveis-123i/base_123i.csv')
df.head()

In [ ]:
len(df)

In [ ]:
# A visual check of each column's value
def check_df_columns(df):
    for col in df.columns:

        print(f" ----- {col} ----- ")
        print(100 * (df[col].value_counts() / df.shape[0]))
        
check_df_columns(df)

In [ ]:
# Print how many nulls on each column
def print_null_cols(df):
    for col in df.columns:
        nulls_value = df[col].isna().sum()
        percentage = 100*(nulls_value / df.shape[0])
        message = "Column {} has {} nulls / {}% ".format(col, nulls_value, percentage)
        print(message)
        
print_null_cols(df)
df.dtypes

In [ ]:
# Function to clean data
def clean_data(df):
    
    # remove weird data with values equal to -1
    df = df[df.maximum_estimate != -1]
    df = df[df.minimum_estimate != -1]
    df = df[df.point_estimate != -1]
    df = df[df.garages != -1]
    df = df[df.rooms != -1]
    df = df[df.useful_area != -1]
    
    # remove cities with low data
    city_dominance = 100 * (df['city'].value_counts() / df.shape[0])
    for a, b in zip(city_dominance.index, city_dominance):
        print(a)
        print(b)
        if b < 1:
            df = df[df.city != a]
            
    # lower case state
    df['state'] = [x.lower() for x in df['state']]
    
    return df

df = clean_data(df)

df = df.reset_index(drop=True)

In [ ]:
print(100 * (df['city'].value_counts() / df.shape[0]))

In [ ]:
len(df)

## Google Maps API para aprimorar o dataset

É possível utilizar o Google Maps API para recuperar o Bairro de cada imóvel utilizando as informações de **Latitude** e **Longitude** fornecidas no dataset. O código abaixo realiza esta extração e insere uma nova coluna no dataframe com o bairros.

**Obs:** O código abaixo não foi rodado devido a necessidade de utilização da API do Google. Para recuperar os bairros da base de dados seria necessário gastar algumas centenas de reais pelo uso da API e por isso fica aqui somente como sugestão de que é possível ser feito.

In [ ]:
"""
neighborhood_list = []
for latitude, longitude in tqdm_notebook(zip(df['latitude'], df['longitude']), total=len(df)):

    #ex: https://maps.googleapis.com/maps/api/geocode/json?key=APIKEY&latlng=-23.56417040,-46.65790930
    
    try:
        response = requests.get('https://maps.googleapis.com/maps/api/geocode/json?key=APIKEY&latlng={0},{1}'.format(latitude, longitude))
        resp_json_payload = response.json()        
        #type_component = resp_json_payload['results'][0]['address_components'][2]['types']
        type_component = resp_json_payload['results'][0]['address_components']
        
        neighborhood = None
        for i in type_component:            
            list_component_type = i['types']            
            if 'sublocality' in list_component_type:            
                neighborhood = i['long_name']
                break
    except:
        neighborhood = None
        
    neighborhood_list.append(neighborhood)

df['neighborhood'] = neighborhood_list
"""

## Encontrando bairros através de Clustering

Considerando que obter as informações sobre o bairro dos imóveis usando uma fonte externa (como o Google Maps API) em alguns casos não seja possível ou desejável, uma outra forma de contornar este problema usando os dados fornecidos pela base é através de *Clustering*!

Podemos clusterizar as informações de Latitude e Longitude para estimar as sublocalidades. Na função abaixo utilizamos o algoritmo de **Kmeans** para prever os bairros dos imóveis nos dados.

In [ ]:
# Create cluster using Kmeans
def find_cluster(df):

    location = df.copy()

    columns_to_keep = ['latitude', 'longitude']

    for col in location:
        if col not in columns_to_keep:
            location = location.drop(col, 1)
    
    location['lng_parsed'] = pd.to_numeric(location['longitude'], errors='coerce')
    location['lat_parsed'] = pd.to_numeric(location['latitude'], errors='coerce')
    
    location = location.drop('longitude', 1)
    location = location.drop('latitude', 1)

    # Remove Outliers do dataframe
    location = location[((location.lat_parsed - location.lat_parsed.mean()) / location.lat_parsed.std()).abs() < 3]
    location = location[((location.lng_parsed - location.lng_parsed.mean()) / location.lng_parsed.std()).abs() < 3]
    location = location[np.abs(location.lat_parsed-location.lat_parsed.mean()) <= (3*location.lat_parsed.std())]
    location = location[np.abs(location.lng_parsed-location.lng_parsed.mean()) <= (3*location.lng_parsed.std())]

    for i in range(0, len(df)):
        if i not in location.index:
            df = df[df.index != i]

    df = df.reset_index(drop=True)

    x1 = location.lng_parsed
    x2 = location.lat_parsed
    
    # Plot charts and execute Kmeans clustering
    plt.figure(num=None, figsize=(8, 6), facecolor='w', edgecolor='k')
    plt.title('Latitude x Longitude')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.scatter(x1, x2)
    plt.show()

    
    plt.figure(num=None, figsize=(8, 6), facecolor='w', edgecolor='k')
    plt.scatter(location.lng_parsed, location.lat_parsed)
    plt.title('Clustering neighborhoods')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')

    location_full = location

    location_np = np.array(location)

    # Execute Kmeans clustering
    # São Paulo has approximately 100 neighborhoods
    k = 100 # Define the value of k
    kmeans = cluster.KMeans(n_clusters=k, random_state=42)
    kmeans.fit(location_np)

    labels = kmeans.labels_
    centroids = kmeans.cluster_centers_

    for i in range(k):
        # select only data observations with cluster label == i
        ds = location_np[np.where(labels==i)]
        # plot the data observations
        plt.plot(ds[:,0],ds[:,1],'o', markersize=6)
        # plot the centroids
        lines = plt.plot(centroids[i,0],centroids[i,1],'kx')
        # make the centroid x's bigger
        plt.setp(lines,ms=8.0)
        plt.setp(lines,mew=3.0)
    plt.show()

    list_areas_kmeans = []
    for f, b in zip(labels, location_full.index):
        list_areas_kmeans.append(f)

    return list_areas_kmeans, df, location

In [ ]:
list_areas_kmeans, df, location = find_cluster(df)

df['area_kmeans'] = list_areas_kmeans

In [ ]:
df.head()

# Informações estretégicas

Podemos utilizar as informações da base para fazer análises sobre coisas interessantes em relação a lógica de négocio do setor imobiliário. Responder algumas perguntas e chegar em algumas conclusões importantes com o uso de dados.

## Variação de preços

Abaixo podemos ver um gráfico estilo Boxplot que mostra a variação de preço dos imóveis (point_estimate) de acordo com as áreas encontradas pela clusterização. 

In [ ]:
order_by_median = df.groupby(by=["area_kmeans"])["point_estimate"].median().sort_values(ascending=True).index

import matplotlib.ticker as ticker

sns.set(rc={'figure.figsize':(17,5)})
sns.set(palette="pastel")
sns.boxplot(x="area_kmeans", y="point_estimate", color="orange", data=df, showfliers=False)
sns.despine(offset=10, trim=True)
ax = plt.gca()
ax.xaxis.set_major_formatter(ticker.FormatStrFormatter('%d'))
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5))
plt.show()

sns.set(rc={'figure.figsize':(17,5)})
sns.set(font_scale=0.78)
sns.boxplot(x="area_kmeans", y="point_estimate", palette="rainbow", data=df, showfliers=False, order=order_by_median)
sns.despine(offset=10, trim=True)
#ax = plt.gca()
#ax.xaxis.set_major_formatter(ticker.FormatStrFormatter('%d'))
#ax.xaxis.set_major_locator(ticker.MultipleLocator(base=5))
plt.show()

Os clusters abaixo são as áreas que possuem a maior média de preço dos imóveis:

In [ ]:
print(order_by_median[-5:].tolist())

Saber quais são as áreas de uma cidade que possuem as maiores ou menores variabilidades nos preços é uma informação bastante importante para empresas que atuam no setor imobiliário. Estratégias de marketing, por exemplo, podem fazer uso dessa informação para elaborar ações em regiões específicas da cidade. Equipes responsáveis pela aquisição de novos imóveis também podem utilizar esta informação para identificar as diferentes realidades das diversas regiões da cidade.

## Heatmap

Podemos utilizar os dados de Latitude e Longitude, junto com as áreas encontradas pelo clustering, para criar um **mapa de calor** onde pode ser visualizado todos os imóveis da base de dados. Conseguimos identificar quais regiões os imóveis da base estão localizados e também quais os clusters que possuem a maior média (marcador vermelho) e menor média (marcador azul) de preço dos imóveis.

In [ ]:
latitude_list = []
longitude_list = []
label_area_kmeans = []
for i in order_by_median[-5:]:
    expensive_areas = df.loc[(df['area_kmeans'] == i)]
    latitude_list.append(expensive_areas['latitude'][0:1])
    longitude_list.append(expensive_areas['longitude'][0:1])
    label_area_kmeans.append(i)
    
latitude_list_less = []
longitude_list_less = []
label_area_kmeans_less = []
for i in order_by_median[0:5]:
    less_expensive_areas = df.loc[(df['area_kmeans'] == i)]
    latitude_list_less.append(less_expensive_areas['latitude'][0:1])
    longitude_list_less.append(less_expensive_areas['longitude'][0:1])
    label_area_kmeans_less.append(i)

In [ ]:
import folium
from folium import plugins

heatmap = df.copy()
heatmap['count'] = 1
base_heatmap = folium.Map(location=['-23.5713874', '-46.6522521'], zoom_start=12)

for a, b, c in zip(latitude_list, longitude_list, label_area_kmeans):    
    folium.Marker((a, b), popup=c, icon=folium.Icon(color='red')).add_to(base_heatmap)
    
for a, b, c in zip(latitude_list_less, longitude_list_less, label_area_kmeans_less):    
    folium.Marker((a, b), popup=c, icon=folium.Icon(color='darkblue')).add_to(base_heatmap)

plugins.HeatMap(data=heatmap[['latitude', 'longitude', 'count']].groupby(['latitude', 'longitude']).sum().reset_index().values.tolist(), radius=8, max_zoom=4).add_to(base_heatmap)

base_heatmap

A visualização dessas informações pode ser bastante importante para uma empresa que deseja, por exemplo, expandir para outras cidades onde não se possui conhecimento tácito sobre o local. Saber previamente qual a variação de preço dos imóveis das subregiões da cidade pode ser uma informação crucial para o sucesso do negócio.

# Modelo

Abaixo é explorado uma possível solução para a precificação de imóveis utilizando machine learning. Foram rodadas versões diferentes do modelo como forma de mostrar alternativas para melhorar os resultados encontrados.

In [ ]:
# Remove unuseful columns
def drop_columns(df, list_columns):
    df = df.drop(list_columns, axis=1)    
    
    return df

list_columns = ['address', 'tower_name', 'latitude', 'longitude', 'city', 'state']
df = drop_columns(df, list_columns)

In [ ]:
# Convert some columns to dummies
def one_hot_encoder(df):
    one_hot = pd.get_dummies(df['building_type'])
    df = df.drop('building_type',axis = 1)
    df = df.join(one_hot)
    
    return df

df = one_hot_encoder(df)

In [ ]:
# Split data/target
def get_data_target(df, target):
    y = df[target]    
    X = df.drop(target,axis = 1)
    
    return X, y

In [ ]:
# Function to train the model
def train(model):

    model = model

    model.fit(X_train, y_train,
            eval_set = [(X_train, y_train), (X_test, y_test)],
            eval_metric = 'rmse',
            early_stopping_rounds = 5,
            verbose=True)

    best_iteration = model.get_booster().best_ntree_limit
    preds = model.predict(X_test, ntree_limit=best_iteration)
    
    return preds, model, best_iteration

In [ ]:
# Function to evaluate some metrics
def evaluate_metrics(preds, model, best_iteration):
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    r2 = r2_score(y_test, preds, multioutput='variance_weighted')
    
    evals_result = model.evals_result()  
    plt.rcParams["figure.figsize"] = (8, 6)
    xgb.plot_importance(model, max_num_features=None)
    plt.show()

    range_evals = np.arange(0, len(evals_result['validation_0']['rmse']))

    val_0 = evals_result['validation_0']['rmse']
    val_1 = evals_result['validation_1']['rmse']
    plt.figure(num=None, figsize=(8, 6), facecolor='w', edgecolor='k')
    plt.plot(range_evals, val_1, range_evals, val_0)
    plt.ylabel('Validation error')
    plt.xlabel('Iteration')
    plt.show()
    
    print("Best iteration: %f" % (int(best_iteration)))
    print("RMSE: %f" % (rmse))
    print("R2: %f" % (r2))
    print('Observed value single sample: {}'.format(y_test[0:1].tolist()[0]))
    print('Predicted value single sample: {}'.format(int(model.predict(X_test[0:1], ntree_limit=best_iteration)[0])))

## Treinamento e previsões

Abaixo rodamos diversas versões do modelo e analisamos os resultados.

<b>Primeiro modelo:</b>

In [ ]:
# First model
target = 'point_estimate'
X, y = get_data_target(df, target)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xg_reg = xgb.XGBRegressor()

preds, model, best_iteration = train(xg_reg)

evaluate_metrics(preds, model, best_iteration)

Analisando os resultados, fica claro que esta versão do modelo acima sofre de **Data Leakage**, pois as variáveis *minimum_estimate* e *maximum_estimate* são altamente correlacionadas com o target utilizado -> *point_estimate*. Isto acontece quando os dados que estamos usando para treinar o algoritmo contém informações do que nós estamos tentando prever. No caso, é possível descobrir o nosso target (point_estimate) simplesmente manipulando as outras variáveis do modelo, sem nem mesmo precisar utilizar Machine Learning.

Por mais que estejamos utilizando *early stopping* para evitar overfitting no modelo, são as proprias features que estão causando este problema. Neste caso, é importante ficar atento, pois, por mais que este modelo esteja aparentemente gerando excelentes previsões com os dados de teste, o modelo não é generalizavel para dados novos.

Logo abaixo rodamos uma outra versão do modelo, agora retirando as features *minimum_estimate* e *maximum_estimate*.

<b>Segundo modelo:</b>

In [ ]:
# Second model
list_columns = ['minimum_estimate', 'maximum_estimate']
df = drop_columns(df, list_columns)

target = 'point_estimate'
X, y = get_data_target(df, target)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xg_reg = xgb.XGBRegressor()

preds, model, best_iteration = train(xg_reg)

evaluate_metrics(preds, model, best_iteration)

Neste modelo anteior, como já era esperado, ocorreu um aumento do erro na métrica RMSE e uma diminuição do R2. Contudo, nos livramos das variáveis que estavam causando Data Leakage e temos um modelo mais confiável e generalizável.

Podemos tentar melhorar mais ainda o modelo utilizando algumas técnicas para aprimoramento das *features*. A função **feature_engineering** gera mais seis novas *features* no dataset para tentar ajudar o modelo a ter uma melhor performance.

Calculamos a média e o desvio padrão dos valores de quartos, garagens e área útil para cada cluster.

<b>Terceiro modelo:</b>

In [ ]:
# Function to create new features
def feature_engineering(df):

    mean_rooms_list = []
    mean_garages_list = []
    mean_useful_area_list = []    
    std_rooms_list = []
    std_garages_list = []
    std_useful_area_list = []
    
    for i in tqdm_notebook(df.index):

        rooms_df = df.loc[(df['rooms']) & ((df['area_kmeans'] ==  df['area_kmeans'][i]))]
        mean_rooms = rooms_df['rooms'].mean()
        mean_rooms_list.append(float(mean_rooms))
        std_rooms = rooms_df['rooms'].std()
        std_rooms_list.append(float(std_rooms))

        garages_df = df.loc[(df['garages']) & ((df['area_kmeans'] ==  df['area_kmeans'][i]))]
        mean_garages = garages_df['garages'].mean()
        mean_garages_list.append(float(mean_garages))
        std_garages = garages_df['garages'].std()
        std_garages_list.append(float(std_garages))
        
        useful_area_df = df.loc[(df['useful_area']) & ((df['area_kmeans'] ==  df['area_kmeans'][i]))]
        mean_useful_area = useful_area_df['useful_area'].mean()
        mean_useful_area_list.append(float(mean_useful_area))
        std_useful_area = useful_area_df['useful_area'].std()
        std_useful_area_list.append(float(std_useful_area))

    df['mean_rooms'] = mean_rooms_list
    df['mean_garages'] = mean_garages_list
    df['mean_useful_area'] = mean_useful_area_list
    df['std_rooms'] = std_rooms_list
    df['std_garages'] = std_garages_list
    df['std_useful_area'] = std_useful_area_list    
    
    return df

In [ ]:
df = feature_engineering(df)
df.head()

In [ ]:
# Third model
target = 'point_estimate'
X, y = get_data_target(df, target)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xg_reg = xgb.XGBRegressor()

preds, model, best_iteration = train(xg_reg)

evaluate_metrics(preds, model, best_iteration)

Vemos que o processo de *feature engineering*, ou seja, gerar novas *features* derivadas das *features* já existentes, fornece um bom resultado. O modelo melhorou e essas novas *features* estão com uma "importância" alta.

## Testando um dataset mais profundo

Certamente um dos caminhos para melhorar o modelo de *valuation* de imóveis seria melhorar a base de dados. A base fornecida possui um número limitado de *features* em relação ao que é possível encontrar dentro da lógica do negócio.

**Por exemplo:** Seria importante incluir mais características em relação aos imóveis, como quantidade de banheiros, suítes, elevadores, piscina, o bairro do imóvel, o valor do condomínio, se o imóvel é novo, se o apartamento está sendo vendido mobiliado, etc.

Quanto mais características em relação aos imóveis, melhor será a performance do algoritmo, e várias características podem ser facilmente obtidas atráves de webscraping ou em parcerias com outras empresas que possuem uma base de dados de imóveis.

Alguns meses atrás eu fiz o upload no Kaggle de um dataset com 13 mil imóveis localizados na cidade de São Paulo. Esses imóveis foram obtidos através do webscraping de diversos sites de imóveis do Brasil.

Apesar deste dataset ser menor em relação ao dataset fornecido para o case, ele possui mais *features*.

Logo abaixo eu rodei o mesmo modelo utilizado acima, mas agora utilizando os dados desta nova base de dados, que possui mais features em relação a base fornecida para o case.

In [ ]:
df = pd.read_csv('../input/sao-paulo-real-estate-sale-rent-april-2019/sao-paulo-properties-april-2019.csv')
df = df[df['Negotiation Type'] == 'sale']
df.head()

<b>Quarto modelo:</b>

In [ ]:
list_columns = ['Property Type', 'Latitude', 'Longitude', 'Negotiation Type']
df = drop_columns(df, list_columns)

#df['District'] = df['District'].astype('category')

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
df['District'] = labelencoder.fit_transform(df['District'])

target = 'Price'
X, y = get_data_target(df, target)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xg_reg = xgb.XGBRegressor()

preds, model, best_iteration = train(xg_reg)

evaluate_metrics(preds, model, best_iteration)

A métrica **RMSE**, que avalia a perfomance do modelo, foi muito melhor com esta nova base de dados. Isto mostra que obter mais *features* em relação aos imóveis melhora o modelo de *valuation*.

## Uma previsão real

São inúmeras as possibilidades de aplicação deste tipo de modelo para o setor imobiliário. Conseguir saber com fundamento em dados o valor de um imóvel de acordo com suas características é de fato algo muito importante.

Abaixo eu aplico o último modelo treinado em um imóvel da própria Loft localizado no bairro Jardim Paulista, utilizando as características apresentadas no site. Conseguimos assim obter uma previsão de preço, que representa um valor teórico dos imóveis com as mesmas características na região.

![title](https://i.imgur.com/K7Tow8m.png)

In [ ]:
sample = pd.DataFrame({'Condo': 1300, 'Size': 100, 'Rooms': 2, 'Toilets': 2, 'Suites': 1, 'Parking': 1, 'Elevator': 1, 'Furnished': 0, 'Swimming Pool': 0, 'New': 0, 'District': 40}, index=[0])

print(model.predict(sample, ntree_limit=best_iteration))

Este tipo de análise pode ser útil tanto para a elaboração do preço final de venda do imóvel, quanto para a aquisição de imóveis. O output de resultados como este, em conjunto com o modelo de negócio, pode acelerar ainda mais a compra instantânea de imóveis.


# Conclusões e sugestões

Este presente estudo procurou explorar a base de dados fornecida pela Loft e extrair informações importantes com implicações para o seu negócio.

Mesmo não possuindo informações sobre os bairros onde os imóveis da base estavam localizados, uma informação bastante importante no mercado imobiliário, foi possível utilizar ténicas de clusterização para obter uma aproximação dessa informação. Esses clusters se mostraram bastante importantes como *feature* (característica) para o modelo desenvolvido.

Descobrimos que mesmo a base possuindo preços máximos e mínimos estimados para os imóveis, estas informações não são úteis para o desenvolvimento de um modelo de machine learning. Porque a utilização dessas informações leva ao problema de Data Leakage, onde um modelo aparenta bons resultados, mas não é generalizável para dados novos.

Vimos que realizar processos de *feature engineering*, onde novas *features* são criadas derivadas das *features* já existentes, é bastante relevante para melhorar as previsões do modelo.

Por fim, apresentei uma sugestão de outra estrutura de base de dados do setor imobiliário, desenvolvida por mim alguns meses atrás, que possui informações mais detalhadas sobre imóveis na região de São Paulo. Esta nova base se apresentou mais eficaz e mostrou melhores resultados quando aplicada ao modelo de *valuation* de imóveis.